In [16]:
%pip install --upgrade tensorflow

                                              0.0/276.5 MB ? eta -:--:--
                                              0.1/276.5 MB 3.2 MB/s eta 0:01:27
                                              0.3/276.5 MB 5.2 MB/s eta 0:00:54
                                             1.2/276.5 MB 11.1 MB/s eta 0:00:25
                                             2.5/276.5 MB 16.1 MB/s eta 0:00:18
                                             2.9/276.5 MB 18.7 MB/s eta 0:00:15
                                             5.4/276.5 MB 23.1 MB/s eta 0:00:12
     -                                       8.1/276.5 MB 30.3 MB/s eta 0:00:09
     -                                      10.5/276.5 MB 40.9 MB/s eta 0:00:07
     -                                      13.0/276.5 MB 54.7 MB/s eta 0:00:05
     --                                     15.9/276.5 MB 65.6 MB/s eta 0:00:04
     --                                     18.3/276.5 MB 65.6 MB/s eta 0:00:04
     --                                     20.

ERROR: Could not install packages due to an OSError: [WinError 5] Accesso negato: 'C:\\Users\\USER\\anaconda3\\Lib\\site-packages\\~ensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
import os

# Definizione dei parametri di caricamento delle immagini
batch_size = 32
image_size = (816, 612)
directory = '../Dataset/JPEGImages/T'
test_split = 0.2

image_files = os.listdir(directory)
# Caricamento delle immagini dalla directory
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    class_names=None,
    batch_size=batch_size,
    validation_split=test_split,
    subset="training",
    seed=123
)

class_names = os.listdir(directory)
print("Nomi delle classi:", class_names)

# Calcolo del numero di campioni per il set di test
num_samples = train_dataset.cardinality().numpy()
num_test_samples = int(num_samples * test_split)

# Divisione del set di addestramento in set di addestramento e test
test_dataset = train_dataset.take(num_test_samples)
train_dataset = train_dataset.skip(num_test_samples)

Found 1477 files belonging to 20 classes.
Using 1182 files for training.
Nomi delle classi: ['Apple', 'Banana', 'Bread', 'Bun', 'Doughnut', 'Egg', 'Fired_dough', 'Grape', 'Lemon', 'Litchi', 'Mango', 'Mix', 'Mooncake', 'Orange', 'Peach', 'Pear', 'Plum', 'Qiwi', 'Sachima', 'Tomato']


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Calcolo del numero di classi
num_classes = len(train_dataset.class_names)

# Preprocessamento delle immagini
preprocess_input = tf.keras.applications.resnet50.preprocess_input

# Creazione del modello R-CNN
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False

input_layer = Input(shape=(224, 224, 3))
preprocessed_input = preprocess_input(input_layer)

rcnn_features = base_model(preprocessed_input)
flatten_features = tf.keras.layers.Flatten()(rcnn_features)
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(flatten_features)

model = Model(inputs=input_layer, outputs=output_layer)

# Compilazione del modello
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Addestramento del modello
model.fit(train_dataset, epochs=10)

AttributeError: '_SkipDataset' object has no attribute 'class_names'